# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from graph_representation_generator.graph_representation_generator import (
    GraphRepresentationGenerator,
)
from dataset_manager.movie_lens_manager import MovieLensManager
from dataset_manager.kg_manager import ROOT
from llm_manager.graph_prompter_hf.classifier import GraphPrompterHF

In [2]:
MODEL_NAME = "google/bert_uncased_L-2_H-128_A-2"
EPOCHS = 10
BATCH_SIZE_LLM = 256

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [3]:
kg_manager = MovieLensManager()

llm_df = kg_manager.llm_df.merge(kg_manager.target_df[["id", "prompt_feature_title", "prompt_feature_genres"]].rename(columns={"id": "target_id"}), on = "target_id")
llm_df

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-32m.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [4]:
kg_manager.data

HeteroData(
  source={ node_id=[610] },
  target={
    node_id=[9742],
    x=[9742, 20],
  },
  (source, edge, target)={ edge_index=[2, 100836] },
  (target, rev_edge, source)={ edge_index=[2, 100836] }
)

Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
graph_representation_generator_graph_prompter_hf = GraphRepresentationGenerator(
    kg_manager.data.to("cuda"),
    kg_manager.gnn_train_data.to("cuda"),
    kg_manager.gnn_val_data.to("cuda"),
    kg_manager.gnn_test_data.to("cuda"),
    hidden_channels=128,
    kge_dimension=128,
    force_recompute=False,
    device="cuda",
)

loading pretrained model
Device: 'cuda'


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [6]:
graph_prompter_hf_bert_classifier = GraphPrompterHF(
    kge_manager=kg_manager,
    get_embeddings_cb=graph_representation_generator_graph_prompter_hf.get_embeddings,
    model_name=MODEL_NAME,
    vanilla_model_path=f"{ROOT}/llm/vanilla/training/best",
    # gnn_parameters=list(
    #     graph_representation_generator_graph_prompter_hf.model.parameters()
    # ),
)

Some weights of GraphPrompterHFBertForSequenceClassification were not initialized from the model checkpoint at ./data/llm/vanilla/training/best and are newly initialized because the shapes did not match:
- bert.embeddings.token_type_embeddings.weight: found shape torch.Size([3, 128]) in the checkpoint and torch.Size([5, 128]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device cuda
5
5
5


In [7]:
dataset_embedding = kg_manager.generate_graph_prompter_hf_embedding_dataset(
    graph_prompter_hf_bert_classifier.tokenizer.sep_token,
    graph_prompter_hf_bert_classifier.tokenizer.pad_token,
    graph_prompter_hf_bert_classifier.tokenize_function,
)

import torch

gnn_model = graph_representation_generator_graph_prompter_hf.model
trainer = graph_prompter_hf_bert_classifier._get_trainer(
    dataset_embedding, epochs=EPOCHS, batch_size=BATCH_SIZE_LLM
)
optimizer = torch.optim.SGD(
    list(trainer.model.parameters()) + list(gnn_model.parameters()),
    lr=0.001,
    momentum=0.9,
)


def inspect_gradients(trainer, batch):
    optimizer.zero_grad()  # Clear previous gradients
    trainer.model.eval()  # Set model to eval mode (no dropout, etc.)
    outputs = trainer.model(**batch)
    loss = outputs.loss
    loss.backward()  # Run backward to compute gradients
    optimizer.step()
    for name, param in gnn_model.named_parameters():
        if param.requires_grad and param.grad is not None:
            print(f"Gradient for gnn {name}: {param.grad}")

    # for name, param in trainer.model.named_parameters():
    #     if param.requires_grad and param.grad is not None:
    #         print(f"Gradient for LLM{name}: {(param.grad > 0).any()}")


# Example of grabbing a batch and inspecting gradients
for i in range(1):
    sample_batch = next(iter(trainer.get_train_dataloader()))

    inspect_gradients(trainer, sample_batch)

tens_1 = graph_representation_generator_graph_prompter_hf.model.to(
    device="cpu"
).state_dict()

tens_2 = torch.load("./data/gnn/backup/model_128.pth")
same = True
for key in tens_1.keys():
    if isinstance(tens_1[key], torch.Tensor):
        if not (tens_1[key] == tens_2[key]).all():
            same = False
print(same)

In [8]:
graph_prompter_hf_bert_classifier.train_model_on_data(
    dataset_embedding,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE_LLM,
)
# graph_representation_generator_graph_prompter_hf.save_model()

  0%|          | 0/2210 [00:00<?, ?it/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.5269, 'grad_norm': 2.356945753097534, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.05}
{'loss': 0.5371, 'grad_norm': 1.5672634840011597, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.09}
{'loss': 0.5213, 'grad_norm': 1.6135259866714478, 'learning_rate': 3e-06, 'epoch': 0.14}
{'loss': 0.5222, 'grad_norm': 1.489190936088562, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.18}
{'loss': 0.5401, 'grad_norm': 1.5681816339492798, 'learning_rate': 5e-06, 'epoch': 0.23}
{'loss': 0.5172, 'grad_norm': 2.9827609062194824, 'learning_rate': 6e-06, 'epoch': 0.27}
{'loss': 0.5017, 'grad_norm': 1.6629126071929932, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.32}
{'loss': 0.5083, 'grad_norm': 1.4409894943237305, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.36}
{'loss': 0.4976, 'grad_norm': 1.8311820030212402, 'learning_rate': 9e-06, 'epoch': 0.41}
{'loss': 0.5188, 'grad_norm': 1.9138226509094238, 'learning_rate': 1e-05, 'epoch': 0.45}
{'loss': 0.4989, 'grad_norm': 

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.4471861720085144, 'eval_accuracy': 0.7947535455717545, 'eval_runtime': 25.024, 'eval_samples_per_second': 805.866, 'eval_steps_per_second': 3.157, 'epoch': 1.0}
{'loss': 0.4708, 'grad_norm': 1.9528943300247192, 'learning_rate': 2.3000000000000003e-05, 'epoch': 1.04}
{'loss': 0.4581, 'grad_norm': 1.5401691198349, 'learning_rate': 2.4e-05, 'epoch': 1.09}
{'loss': 0.4641, 'grad_norm': 2.118473768234253, 'learning_rate': 2.5e-05, 'epoch': 1.13}
{'loss': 0.4826, 'grad_norm': 1.5756334066390991, 'learning_rate': 2.6000000000000002e-05, 'epoch': 1.18}
{'loss': 0.4911, 'grad_norm': 1.8010320663452148, 'learning_rate': 2.7000000000000002e-05, 'epoch': 1.22}
{'loss': 0.4987, 'grad_norm': 1.6645898818969727, 'learning_rate': 2.8000000000000003e-05, 'epoch': 1.27}
{'loss': 0.4798, 'grad_norm': 1.5093493461608887, 'learning_rate': 2.9e-05, 'epoch': 1.31}
{'loss': 0.4719, 'grad_norm': 1.4097771644592285, 'learning_rate': 3e-05, 'epoch': 1.36}
{'loss': 0.4622, 'grad_norm': 1.837345361

...

{'eval_loss': 0.40961503982543945, 'eval_accuracy': 0.8202419914707925, 'eval_runtime': 25.3515, 'eval_samples_per_second': 795.457, 'eval_steps_per_second': 3.116, 'epoch': 8.0}
{'loss': 0.3542, 'grad_norm': 1.1746166944503784, 'learning_rate': 1.2865497076023392e-05, 'epoch': 8.01}
{'loss': 0.352, 'grad_norm': 1.1073132753372192, 'learning_rate': 1.2573099415204679e-05, 'epoch': 8.05}
{'loss': 0.338, 'grad_norm': 0.9531964063644409, 'learning_rate': 1.2280701754385964e-05, 'epoch': 8.1}
{'loss': 0.375, 'grad_norm': 1.139713168144226, 'learning_rate': 1.1988304093567252e-05, 'epoch': 8.14}
{'loss': 0.3465, 'grad_norm': 1.0720798969268799, 'learning_rate': 1.1695906432748537e-05, 'epoch': 8.19}
{'loss': 0.3342, 'grad_norm': 1.094533085823059, 'learning_rate': 1.1403508771929824e-05, 'epoch': 8.24}
{'loss': 0.3631, 'grad_norm': 1.5714579820632935, 'learning_rate': 1.1111111111111112e-05, 'epoch': 8.28}
{'loss': 0.3298, 'grad_norm': 0.8268879055976868, 'learning_rate': 1.0818713450292397

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.4265603721141815, 'eval_accuracy': 0.8138450857879599, 'eval_runtime': 25.0909, 'eval_samples_per_second': 803.718, 'eval_steps_per_second': 3.149, 'epoch': 9.0}
{'loss': 0.3393, 'grad_norm': 1.524579405784607, 'learning_rate': 6.432748538011696e-06, 'epoch': 9.0}
{'loss': 0.3541, 'grad_norm': 1.1281825304031372, 'learning_rate': 6.140350877192982e-06, 'epoch': 9.05}
{'loss': 0.3492, 'grad_norm': 0.9135451316833496, 'learning_rate': 5.8479532163742686e-06, 'epoch': 9.1}
{'loss': 0.3435, 'grad_norm': 1.3494091033935547, 'learning_rate': 5.555555555555556e-06, 'epoch': 9.14}
{'loss': 0.3589, 'grad_norm': 0.952190637588501, 'learning_rate': 5.263157894736842e-06, 'epoch': 9.19}
{'loss': 0.3524, 'grad_norm': 1.0929826498031616, 'learning_rate': 4.970760233918129e-06, 'epoch': 9.23}
{'loss': 0.3371, 'grad_norm': 1.3541598320007324, 'learning_rate': 4.678362573099415e-06, 'epoch': 9.28}
{'loss': 0.3371, 'grad_norm': 0.9167181253433228, 'learning_rate': 4.3859649122807014e-06,

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.4146804213523865, 'eval_accuracy': 0.8206386987999603, 'eval_runtime': 25.2269, 'eval_samples_per_second': 799.386, 'eval_steps_per_second': 3.132, 'epoch': 10.0}
{'train_runtime': 1111.2356, 'train_samples_per_second': 508.164, 'train_steps_per_second': 1.989, 'train_loss': 0.38526441144727475, 'epoch': 10.0}


In [9]:
import torch

tens_1 = graph_representation_generator_graph_prompter_hf.model.to(
    device="cpu"
).state_dict()

tens_2 = torch.load("./data/gnn/backup/model_128.pth")
same = True
for key in tens_1.keys():
    if isinstance(tens_1[key], torch.Tensor):
        if not (tens_1[key] == tens_2[key]).all():
            same = False
print(same)

True
